##  taken from: https://machinelearningmastery.com/how-to-develop-a-pix2pix-gan-for-image-to-image-translation/

In [1]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from matplotlib import pyplot

In [2]:
# define the discriminator model
def define_discriminator(image_shape):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_src_image = Input(shape=image_shape)
    # target image input
    in_target_image = Input(shape=image_shape)
    # concatenate images channel-wise
    merged = Concatenate()([in_src_image, in_target_image])
    # C64
    d = Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv2D(512, (4, 4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv2D(1, (4, 4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target_image], patch_out)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model


# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv2D(n_filters, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g


# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add upsampling layer
    g = Conv2DTranspose(n_filters, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(layer_in)
    # add batch normalization
    g = BatchNormalization()(g, training=True)
    # conditionally add dropout
    if dropout:
        g = Dropout(0.5)(g, training=True)
    # merge with skip connection
    g = Concatenate()([g, skip_in])
    # relu activation
    g = Activation('relu')(g)
    return g


# define the standalone generator model
def define_generator(image_shape=(256, 256, 3)):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=image_shape)
    # encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    # decoder model
    d1 = decoder_block(b, e7, 512)
    d2 = decoder_block(d1, e6, 512)
    d3 = decoder_block(d2, e5, 512)
    d4 = decoder_block(d3, e4, 512, dropout=False)
    d5 = decoder_block(d4, e3, 256, dropout=False)
    d6 = decoder_block(d5, e2, 128, dropout=False)
    d7 = decoder_block(d6, e1, 64, dropout=False)
    # output
    g = Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('tanh')(g)
    # define model
    model = Model(in_image, out_image)
    return model


# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    # connect the source image to the generator input
    gen_out = g_model(in_src)
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out])
    # src image as input, generated image and classification output
    model = Model(in_src, [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1, 100])
    return model


# load and prepare training images
def load_real_samples(filename):
    # load compressed arrays
    data = load(filename)
    # unpack arrays
    X1, X2 = data['arr_0'], data['arr_1']
    # scale from [0,255] to [-1,1]
    X1 = (X1 - 127.5) / 127.5
    X2 = (X2 - 127.5) / 127.5
    return [X2, X1]


# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
    # unpack dataset
    trainA, trainB = dataset
    # choose random instances
    ix = randint(0, trainA.shape[0], n_samples)
    # retrieve selected images
    X1, X2 = trainA[ix], trainB[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, patch_shape, patch_shape, 1))
    return [X1, X2], y


# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
    # generate fake instance
    X = g_model.predict(samples)
    # create 'fake' class labels (0)
    y = zeros((len(X), patch_shape, patch_shape, 1))
    return X, y


# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=3):
    # select a sample of input images
    [X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
    # generate a batch of fake samples
    X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
    # scale all pixels from [-1,1] to [0,1]
    X_realA = (X_realA + 1) / 2.0
    X_realB = (X_realB + 1) / 2.0
    X_fakeB = (X_fakeB + 1) / 2.0
    # plot real source images
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X_realA[i])
    # plot generated target image
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(X_fakeB[i])
    # plot real target image
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples * 2 + i)
        pyplot.axis('off')
        pyplot.imshow(X_realB[i])
    # save plot to file
    filename1 = 'plot_%06d.png' % (step + 1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = 'model_%06d.h5' % (step + 1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))


# train pix2pix models
def train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=1):
    print("n_epochs: ", n_epochs)
    # determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]
    # unpack dataset
    trainA, trainB = dataset
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(trainA) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # manually enumerate epochs
    for i in range(n_steps):
        # select a batch of real samples
        [X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
        # generate a batch of fake samples
        X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
        # summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i + 1, d_loss1, d_loss2, g_loss))
        # summarize model performance
        if (i + 1) % (bat_per_epo * 10) == 0:
            summarize_performance(i, g_model, dataset)

In [3]:
def train_model(button):
    # load image data
    dataset = load_real_samples('maps_256.npz')
    print('Loaded', dataset[0].shape, dataset[1].shape)
    # define input shape based on the loaded dataset
    image_shape = dataset[0].shape[1:]
    # define the models
    d_model = define_discriminator(image_shape)
    g_model = define_generator(image_shape)
    # define the composite model
    gan_model = define_gan(g_model, d_model, image_shape)
    # train model
    train(d_model, g_model, gan_model, dataset, n_epochs=iterations.value)
    

In [4]:
from ipywidgets import widgets
iterations = widgets.IntText(value=100, description='epochs:', disabled=False)
submit = widgets.Button(description='Submit')
submit.on_click(train_model)
widgets.HBox([iterations, submit])

Loaded (1096, 256, 256, 3) (1096, 256, 256, 3)
n_epochs:  1
>1, d1[0.365] d2[0.764] g[85.978]
>2, d1[0.289] d2[0.716] g[72.941]
>3, d1[0.319] d2[0.644] g[64.384]
>4, d1[0.388] d2[0.482] g[77.770]
>5, d1[0.315] d2[0.566] g[51.150]
>6, d1[0.384] d2[0.480] g[69.786]
>7, d1[0.275] d2[0.387] g[70.479]
>8, d1[0.313] d2[0.409] g[51.994]
>9, d1[0.194] d2[0.322] g[36.919]
>10, d1[0.198] d2[0.248] g[63.155]
>11, d1[0.176] d2[0.201] g[54.065]
>12, d1[0.120] d2[0.164] g[57.817]
>13, d1[0.129] d2[0.204] g[31.010]
>14, d1[0.140] d2[0.139] g[53.003]
>15, d1[0.344] d2[0.991] g[42.691]
>16, d1[0.126] d2[0.668] g[37.595]
>17, d1[0.270] d2[0.099] g[30.985]
>18, d1[0.105] d2[0.120] g[24.618]
>19, d1[0.185] d2[0.083] g[35.537]
>20, d1[0.045] d2[0.091] g[42.500]
>21, d1[0.047] d2[0.055] g[34.986]
>22, d1[0.043] d2[0.069] g[38.398]
>23, d1[0.420] d2[2.120] g[25.455]
>24, d1[0.049] d2[0.988] g[31.481]
>25, d1[0.208] d2[0.375] g[28.157]
>26, d1[0.482] d2[0.285] g[27.081]
>27, d1[0.388] d2[0.240] g[28.285]
>28,

>232, d1[0.494] d2[0.176] g[19.248]
>233, d1[0.014] d2[0.259] g[12.264]
>234, d1[0.027] d2[0.103] g[15.912]
>235, d1[0.014] d2[0.059] g[23.167]
>236, d1[0.080] d2[0.060] g[13.288]
>237, d1[0.416] d2[1.404] g[15.393]
>238, d1[0.192] d2[0.321] g[15.315]
>239, d1[0.105] d2[0.089] g[20.176]
>240, d1[0.179] d2[0.083] g[16.209]
>241, d1[0.069] d2[0.217] g[14.869]
>242, d1[0.869] d2[0.093] g[18.086]
>243, d1[0.053] d2[0.176] g[19.614]
>244, d1[0.113] d2[0.208] g[12.542]
>245, d1[0.361] d2[0.311] g[11.152]
>246, d1[0.034] d2[0.103] g[20.068]
>247, d1[0.173] d2[0.231] g[17.213]
>248, d1[0.032] d2[0.070] g[14.126]
>249, d1[0.048] d2[0.059] g[13.855]
>250, d1[1.032] d2[0.074] g[21.452]
>251, d1[0.062] d2[0.105] g[16.237]
>252, d1[0.341] d2[0.427] g[11.660]
>253, d1[0.014] d2[0.084] g[23.084]
>254, d1[0.041] d2[0.071] g[22.844]
>255, d1[0.925] d2[0.145] g[13.027]
>256, d1[0.047] d2[0.144] g[14.938]
>257, d1[0.034] d2[0.198] g[13.643]
>258, d1[0.017] d2[0.062] g[19.539]
>259, d1[0.558] d2[0.138] g[

>461, d1[0.214] d2[0.194] g[12.230]
>462, d1[0.068] d2[0.419] g[11.419]
>463, d1[0.409] d2[1.126] g[15.683]
>464, d1[0.818] d2[0.059] g[11.957]
>465, d1[0.086] d2[0.156] g[12.543]
>466, d1[0.068] d2[0.101] g[19.446]
>467, d1[0.035] d2[0.083] g[15.638]
>468, d1[0.715] d2[0.151] g[16.037]
>469, d1[0.026] d2[0.215] g[16.487]
>470, d1[0.023] d2[0.094] g[16.350]
>471, d1[0.333] d2[0.167] g[14.377]
>472, d1[0.114] d2[0.111] g[14.817]
>473, d1[0.736] d2[0.154] g[10.188]
>474, d1[0.033] d2[0.212] g[10.535]
>475, d1[0.713] d2[0.177] g[14.705]
>476, d1[0.006] d2[0.447] g[11.483]
>477, d1[0.209] d2[0.225] g[10.512]
>478, d1[0.040] d2[0.051] g[10.994]
>479, d1[0.014] d2[0.052] g[13.900]
>480, d1[0.085] d2[0.058] g[11.143]
>481, d1[0.221] d2[0.105] g[9.167]
>482, d1[0.004] d2[0.181] g[14.057]
>483, d1[0.005] d2[0.086] g[13.004]
>484, d1[0.558] d2[0.347] g[10.232]
>485, d1[0.273] d2[0.109] g[9.950]
>486, d1[0.192] d2[0.191] g[10.066]
>487, d1[0.218] d2[0.723] g[13.994]
>488, d1[0.060] d2[0.117] g[10

>690, d1[0.265] d2[0.540] g[9.215]
>691, d1[0.089] d2[0.117] g[11.578]
>692, d1[0.020] d2[0.119] g[12.443]
>693, d1[0.230] d2[0.077] g[10.741]
>694, d1[0.054] d2[0.478] g[12.173]
>695, d1[0.526] d2[0.080] g[10.349]
>696, d1[0.919] d2[1.027] g[13.152]
>697, d1[0.045] d2[0.312] g[17.137]
>698, d1[0.432] d2[0.242] g[10.042]
>699, d1[0.007] d2[0.436] g[12.914]
>700, d1[0.074] d2[0.024] g[14.313]
>701, d1[0.591] d2[0.120] g[11.463]
>702, d1[1.539] d2[0.236] g[14.756]
>703, d1[0.360] d2[0.753] g[10.528]
>704, d1[0.008] d2[0.089] g[19.660]
>705, d1[0.072] d2[0.067] g[18.429]
>706, d1[0.014] d2[0.079] g[13.943]
>707, d1[0.017] d2[0.101] g[14.024]
>708, d1[0.458] d2[0.186] g[10.602]
>709, d1[0.083] d2[0.327] g[12.357]
>710, d1[0.581] d2[0.471] g[13.581]
>711, d1[1.281] d2[0.475] g[23.572]
>712, d1[0.056] d2[0.072] g[13.355]
>713, d1[0.084] d2[0.054] g[25.111]
>714, d1[0.713] d2[0.310] g[12.198]
>715, d1[0.157] d2[0.221] g[9.598]
>716, d1[0.183] d2[0.267] g[13.554]
>717, d1[0.496] d2[0.120] g[9.

>919, d1[0.007] d2[0.127] g[16.594]
>920, d1[0.005] d2[0.115] g[12.357]
>921, d1[0.305] d2[0.347] g[7.453]
>922, d1[0.457] d2[0.210] g[9.465]
>923, d1[0.042] d2[0.326] g[10.509]
>924, d1[0.273] d2[0.210] g[10.453]
>925, d1[0.007] d2[0.055] g[22.305]
>926, d1[0.293] d2[0.138] g[9.225]
>927, d1[0.002] d2[0.192] g[13.016]
>928, d1[0.285] d2[0.454] g[8.166]
>929, d1[0.008] d2[0.087] g[10.873]
>930, d1[0.419] d2[0.376] g[9.208]
>931, d1[0.011] d2[0.057] g[11.557]
>932, d1[0.635] d2[0.231] g[11.988]
>933, d1[0.010] d2[0.126] g[11.749]
>934, d1[1.252] d2[0.204] g[12.053]
>935, d1[0.454] d2[0.219] g[8.673]
>936, d1[0.002] d2[0.167] g[29.544]
>937, d1[0.012] d2[0.147] g[11.571]
>938, d1[0.019] d2[0.084] g[26.526]
>939, d1[0.328] d2[0.169] g[12.554]
>940, d1[0.136] d2[0.263] g[9.635]
>941, d1[0.330] d2[0.241] g[8.877]
>942, d1[0.158] d2[0.705] g[13.315]
>943, d1[0.630] d2[0.143] g[8.659]
>944, d1[0.003] d2[0.117] g[12.970]
>945, d1[0.074] d2[0.123] g[10.235]
>946, d1[0.212] d2[0.107] g[9.479]
>9